In [8]:
import torch
import torch.nn as nn
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch_geometric.loader import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
import numpy as np

In [9]:
class GCN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_size, hidden_size)
        self.conv2 = GCNConv(hidden_size, hidden_size)
        self.lin = nn.Linear(hidden_size, output_size)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = global_mean_pool(x, batch)
        x = self.lin(x)
        return x

In [10]:
input_size = 7
hidden_size = 512
output_size = 7
epochs = 100
learning_rate = 1e-3
graph_name = "cb_nabil"
batch_size = 32

In [11]:
graphs = torch.load(f"../graphs/{graph_name}.pt", weights_only=False)
train, val = train_test_split(graphs, test_size=0.2, random_state=12)
train_graphs = DataLoader(train, batch_size=batch_size, shuffle=True)
val_graphs = DataLoader(val, batch_size=batch_size, shuffle=False)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GCN(input_size, hidden_size, output_size)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
ls_fn = nn.MSELoss()

model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in train_graphs:
        batch = batch.to(device)
        optimizer.zero_grad()
        output = model(batch.x, batch.edge_index, batch.batch)
        output = output.view(-1)
        loss = ls_fn(output, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss:.4f}")

model.eval()
predictions = []
ground_truth = []
for batch in val_graphs:
    batch = batch.to(device)
    with torch.no_grad():
        output = model(batch.x, batch.edge_index, batch.batch)
        output = output.view(-1)
        predictions.append(output.cpu().numpy())
        ground_truth.append(batch.y.cpu().numpy())

predictions = np.concatenate(predictions, axis=0)
ground_truth = np.concatenate(ground_truth, axis=0)

mse = mean_squared_error(ground_truth, predictions)
mae = mean_absolute_error(ground_truth, predictions)

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")


Epoch 1/100, Loss: 13.0077
Epoch 2/100, Loss: 8.0074
Epoch 3/100, Loss: 7.8609
Epoch 4/100, Loss: 7.6944
Epoch 5/100, Loss: 7.5884
Epoch 6/100, Loss: 7.3863
Epoch 7/100, Loss: 7.1976
Epoch 8/100, Loss: 7.2755
Epoch 9/100, Loss: 7.1309
Epoch 10/100, Loss: 7.1126
Epoch 11/100, Loss: 7.3039
Epoch 12/100, Loss: 7.0785
Epoch 13/100, Loss: 7.0631
Epoch 14/100, Loss: 6.9379
Epoch 15/100, Loss: 7.2226
Epoch 16/100, Loss: 6.8739
Epoch 17/100, Loss: 6.8067
Epoch 18/100, Loss: 6.7547
Epoch 19/100, Loss: 6.8651
Epoch 20/100, Loss: 6.8160
Epoch 21/100, Loss: 6.6562
Epoch 22/100, Loss: 6.5936
Epoch 23/100, Loss: 6.6391
Epoch 24/100, Loss: 6.6666
Epoch 25/100, Loss: 6.5129
Epoch 26/100, Loss: 6.4145
Epoch 27/100, Loss: 6.6531
Epoch 28/100, Loss: 6.5110
Epoch 29/100, Loss: 6.4172
Epoch 30/100, Loss: 6.4415
Epoch 31/100, Loss: 6.3495
Epoch 32/100, Loss: 6.4161
Epoch 33/100, Loss: 7.0376
Epoch 34/100, Loss: 6.3843
Epoch 35/100, Loss: 6.3478
Epoch 36/100, Loss: 6.4562
Epoch 37/100, Loss: 6.2348
Epoch 38/